In [1]:
%load_ext autoreload
%load_ext autotime

time: 551 µs (started: 2023-01-04 20:59:45 -08:00)


In [2]:
import sys
sys.path.append("../")

import warnings
warnings.filterwarnings('ignore')

time: 788 µs (started: 2023-01-04 20:59:45 -08:00)


In [3]:
%autoreload 2

from vani.recorder import RecorderAnalyzer

# log_dir = "/p/vast1/iopp/recorder_app_logs/cm1/nodes-32/workflow-4/_parquet"
# log_dir = "/p/vast1/iopp/recorder_app_logs/hacc/nodes-32/workflow-0/_parquet"
log_dir = "/p/vast1/iopp/recorder_app_logs/montage/nodes-32/_parquet"
# log_dir = "/p/gpfs1/iopp/recorder_app_logs/genome_pegasus/nodes-32/_parquet"
# log_dir = "/p/gpfs1/iopp/recorder_app_logs/lbann-jag/nodes-32/_parquet"

recorder_analyzer = RecorderAnalyzer(
    cluster_manager_args=dict(
        force_local=True,
        cluster_settings=dict(
            cores=8,
            dashboard_port=3666,
            local_directory="/var/tmp/dask-recorder",
            log_file="%J.log",
            worker_queue="pdebug"
        )
    ),
    working_dir='.recorder',
    debug=True
) 

recorder_analyzer

time: 9.46 s (started: 2023-01-04 20:59:45 -08:00)


In [4]:
recorder_analyzer.cluster_manager.clients

{'local': <Client: 'tcp://192.168.131.2:38003' processes=8 threads=40>,
 'trange': <Client: 'tcp://192.168.131.2:42788' processes=8 threads=40>,
 'file_id': <Client: 'tcp://192.168.131.2:42386' processes=8 threads=40>,
 'proc_id': <Client: 'tcp://192.168.131.2:33110' processes=8 threads=40>}

time: 4.79 ms (started: 2023-01-04 20:59:55 -08:00)


In [5]:
res = recorder_analyzer.analyze_parquet(log_dir=log_dir)

hlm trange 45
hlm file_id 223
hlm proc_id 202
perm ('trange', 'file_id') 223
perm ('trange', 'proc_id') 202
perm ('file_id', 'trange') 43
perm ('file_id', 'proc_id') 175
perm ('proc_id', 'trange') 45
perm ('proc_id', 'file_id') 219
llc ('trange', 'file_id') 223 624
llc ('trange', 'proc_id') 202 606
llc ('file_id', 'trange') 43 91
llc ('file_id', 'proc_id') 175 525
llc ('proc_id', 'trange') 45 97
llc ('proc_id', 'file_id') 219 612
llc-alt ('trange', 'file_id', 'proc_id') 223 219
llc-alt ('trange', 'proc_id', 'file_id') 202 1891
llc-alt ('file_id', 'trange', 'proc_id') 43 255
llc-alt ('file_id', 'proc_id', 'trange') 175 47
llc-alt ('proc_id', 'trange', 'file_id') 45 1921
llc-alt ('proc_id', 'file_id', 'trange') 219 47
time: 30.5 s (started: 2023-01-04 20:59:55 -08:00)


In [6]:
llc, llc_at = res

time: 832 µs (started: 2023-01-04 21:00:25 -08:00)


In [7]:
llc[list(llc.keys())[0]]

acc_pat         app        bw            \
                               max min nunique       fmt       per   
file_id             io_cat                                           
7701937538982921416 1            0   0       1  0.71GB/s  0.002788   
7639450094153155784 1            0   0       2  4.89GB/s  0.019180   
7668160541777642696 1            0   0       1  0.72GB/s  0.002813   
7707004088563713224 1            0   0       1  0.82GB/s  0.003234   
7703344913866474696 1            0   0       1  0.87GB/s  0.003406   
...                            ...  ..     ...       ...       ...   
7646768442325907652 2            0   0       1  0.01GB/s  0.000038   
7665908740742232260 2            0   0       1  0.01GB/s  0.000038   
7645924017395775684 2            0   0       1  0.01GB/s  0.000043   
7674352990043551940 2            0   0       1  0.01GB/s  0.000044   
7703344912644749508 2            0   0       1  0.01GB/s  0.000044   

                                              duration           file_id  \
                                     sum           per       sum nunique   
file_id             io_cat                                                 
7701937538982921416 1       7.637212e+08  1.200895e-01  3.582343       1   
7639450094153155784 1       5.253948e+09  6.036950e-02  1.800858       1   
7668160541777642696 1       7.705880e+08  4.827509e-02  1.440075       1   
7707004088563713224 1       8.857923e+08  3.432751e-02  1.024010       1   
7703344913866474696 1       9.331296e+08  3.287940e-02  0.980812       1   
...                                  ...           ...       ...     ...   
7646768442325907652 2       1.050000e+07  2.681810e-07  0.000008       1   
7665908740742232260 2       1.050000e+07  2.681810e-07  0.000008       1   
7645924017395775684 2       1.166667e+07  2.413629e-07  0.000007       1   
7674352990043551940 2       1.217391e+07  2.313061e-07  0.000007       1   
7703344912644749508 2       1.217391e+07  2.313061e-07  0.000007       1   

                           hostname  ... proc_id rank                size  \
                            nunique  ... nunique  max min nunique     fmt   
file_id             io_cat           ...                                    
7701937538982921416 1             1  ...       1    0   0       1  2.55GB   
7639450094153155784 1             2  ...       2    0   0       1  8.81GB   
7668160541777642696 1             2  ...       2    0   0       1  1.03GB   
7707004088563713224 1             1  ...       1    0   0       1  0.84GB   
7703344913866474696 1             1  ...       1    0   0       1  0.85GB   
...                             ...  ...     ...  ...  ..     ...     ...   
7646768442325907652 2             1  ...       1    0   0       1  0.00GB   
7665908740742232260 2             1  ...       1    0   0       1  0.00GB   
7645924017395775684 2             1  ...       1    0   0       1  0.00GB   
7674352990043551940 2             1  ...       1    0   0       1  0.00GB   
7703344912644749508 2             1  ...       1    0   0       1  0.00GB   

                                                        xfer                
                                     per         sum     max    mean   min  
file_id             io_cat                                                  
7701937538982921416 1       8.857225e-02  2735911684   ~64KB   ~64KB  <4KB  
7639450094153155784 1       3.063098e-01  9461616388  ~256KB  ~256KB  <4KB  
7668160541777642696 1       3.592551e-02  1109704616   ~16KB   ~16KB  <4KB  
7707004088563713224 1       2.936512e-02   907060444   ~16KB   ~16KB  <4KB  
7703344913866474696 1       2.962944e-02   915225052   ~16KB   ~16KB  <4KB  
...                                  ...         ...     ...     ...   ...  
7646768442325907652 2       2.719411e-09          84    <4KB    <4KB  <4KB  
7665908740742232260 2       2.719411e-09          84    <4KB    <4KB  <4KB  
7645924017395775684 2       2.719411e-09          84   

time: 77.3 ms (started: 2023-01-04 21:00:26 -08:00)


In [8]:
llc[list(llc.keys())[1]]

acc_pat         app        bw            \
                               max min nunique       fmt       per   
proc_id             io_cat                                           
9060729382066249156 1            0   0       1  0.71GB/s  0.001278   
5901139297808997828 1            0   0       1  4.89GB/s  0.008789   
1029742319298993604 1            0   0       1  0.82GB/s  0.001482   
2463791410776694212 1            0   0       1  0.87GB/s  0.001561   
244024996488338884  1            0   0       1  2.07GB/s  0.003723   
...                            ...  ..     ...       ...       ...   
244022999328542004  2            0   0       1  0.04GB/s  0.000078   
7825773641328545076 2            0   0       1  0.04GB/s  0.000078   
6424898516467895604 2            0   0       1  0.04GB/s  0.000079   
8716199904583666996 2            0   0       1  0.04GB/s  0.000080   
2402798820724362548 2            0   0       1  0.05GB/s  0.000098   

                                              duration           file_id  \
                                     sum           per       sum nunique   
proc_id             io_cat                                                 
9060729382066249156 1       7.637212e+08  1.010629e-01  3.582343       1   
5901139297808997828 1       5.253973e+09  5.080448e-02  1.800849       1   
1029742319298993604 1       8.857923e+08  2.888876e-02  1.024010       1   
2463791410776694212 1       9.331296e+08  2.767009e-02  0.980812       1   
244024996488338884  1       2.225565e+09  2.662845e-02  0.943890       1   
...                                  ...           ...       ...     ...   
244022999328542004  2       4.639640e+07  3.131465e-07  0.000011       1   
7825773641328545076 2       4.681818e+07  3.103254e-07  0.000011       1   
6424898516467895604 2       4.724771e+07  3.075043e-07  0.000011       1   
8716199904583666996 2       4.768518e+07  3.046831e-07  0.000011       1   
2402798820724362548 2       5.852273e+07  2.482603e-07  0.000009       1   

                           hostname  ... proc_id rank                size  \
                            nunique  ... nunique  max min nunique     fmt   
proc_id             io_cat           ...                                    
9060729382066249156 1             1  ...       1    0   0       1  2.55GB   
5901139297808997828 1             1  ...       1    0   0       1  8.81GB   
1029742319298993604 1             1  ...       1    0   0       1  0.84GB   
2463791410776694212 1             1  ...       1    0   0       1  0.85GB   
244024996488338884  1             1  ...       1    0   0       1  1.96GB   
...                             ...  ...     ...  ...  ..     ...     ...   
244022999328542004  2             1  ...       1    0   0       1  0.00GB   
7825773641328545076 2             1  ...       1    0   0       1  0.00GB   
6424898516467895604 2             1  ...       1    0   0       1  0.00GB   
8716199904583666996 2             1  ...       1    0   0       1  0.00GB   
2402798820724362548 2             1  ...       1    0   0       1  0.00GB   

                                                        xfer                
                                     per         sum     max    mean   min  
proc_id             io_cat                                                  
9060729382066249156 1       7.217528e-02  2735911684   ~64KB   ~64KB  <4KB  
5901139297808997828 1       2.496040e-01  9461613508  ~256KB  ~256KB  <4KB  
1029742319298993604 1       2.392890e-02   907060444   ~16KB   ~16KB  <4KB  
2463791410776694212 1       2.414428e-02   915225052   ~16KB   ~16KB  <4KB  
244024996488338884  1       5.541764e-02  2100688276   ~64KB   ~64KB  <4KB  
...                                  ...         ...     ...     ...   ...  
244022999328542004  2       1.358606e-08         515    <4KB    <4KB  <4KB  
7825773641328545076 2       1.358606e-08         515    <4KB    <4KB  <4KB  
6424898516467895604 2       1.358606e-08         515   

time: 235 ms (started: 2023-01-04 21:00:26 -08:00)


In [9]:
llc[list(llc.keys())[2]]

acc_pat         app         bw                              \
                  max min nunique        fmt           per           sum   
trange io_cat                                                              
1      1            0   0       4   0.64GB/s  3.393910e-03  6.875395e+08   
       2            0   0       3   0.37GB/s  1.961750e-03  3.974120e+08   
2      1            0   0       4   1.31GB/s  6.946883e-03  1.407302e+09   
3      1            0   0       3   0.89GB/s  4.727010e-03  9.575994e+08   
4      1            0   0       4   1.00GB/s  5.301499e-03  1.073980e+09   
...               ...  ..     ...        ...           ...           ...   
23     3            1   0       1  -0.00GB/s -2.254453e-08 -4.567079e+03   
45     3            0   0       1   0.00GB/s  0.000000e+00  0.000000e+00   
21     2            0   0       1   7.12GB/s  3.773577e-02  7.644525e+09   
23     1            0   0       1   2.91GB/s  1.544043e-02  3.127927e+09   
45     2            0   0       1   7.04GB/s  3.730216e-02  7.556684e+09   

               duration            file_id hostname  ... proc_id rank      \
                    per        sum nunique  nunique  ... nunique  max min   
trange io_cat                                        ...                    
1      1       0.287372  10.212173     589       32  ...     202   31   0   
       2       0.120163   4.270150     107       31  ...     160   31   0   
2      1       0.104139   3.700723     247       31  ...      81   25   0   
3      1       0.062299   2.213886     432       32  ...      73    0   0   
4      1       0.056186   1.996636     184       32  ...      64   27   0   
...                 ...        ...     ...      ...  ...     ...  ...  ..   
23     3       0.000074   0.002627       9        2  ...       2    0   0   
45     3       0.000061   0.002153       2        1  ...       1    0   0   
21     2       0.000032   0.001143       2        2  ...       2    0   0   
23     1       0.000031   0.001118       6        2  ...       2    0   0   
45     2       0.000016   0.000565       1        1  ...       1    0   0   

                          size                              xfer                
              nunique      fmt           per         sum     max   mean    min  
trange io_cat                                                                   
1      1           13   6.54GB  1.855074e-01  7021272918  ~256KB  ~16KB   <4KB  
       2           13   1.58GB  4.483629e-02  1697009075  ~256KB  ~64KB   <4KB  
2      1            6   4.85GB  1.376003e-01  5208034870  ~256KB  ~16KB   <4KB  
3      1            1   1.97GB  5.601245e-02  2120015764  ~256KB  ~64KB   <4KB  
4      1            5   2.00GB  5.665527e-02  2144345841  ~256KB  ~16KB   <4KB  
...               ...      ...           ...         ...     ...    ...    ...  
23     3            1  -0.00GB -3.170493e-10         -12    <4KB   <4KB   <4KB  
45     3            1   0.00GB  0.000000e+00           0    <4KB   <4KB   <4KB  
21     2            1   0.01GB  2.308566e-04     8737692   ~16MB  ~16MB  ~16MB  
23     1            1   0.00GB  9.238578e-05     3496710   ~64KB  ~64KB   <4KB  
45     2            1   0.00GB  1.127842e-04     4268771   ~16MB  ~16MB  ~16MB  

[91 rows x 24 columns]

time: 61.6 ms (started: 2023-01-04 21:00:26 -08:00)


In [10]:
llc[list(llc.keys())[3]]

acc_pat         app        bw            \
                               max min nunique       fmt       per   
proc_id             io_cat                                           
9060729382066249156 1            0   0       1  0.71GB/s  0.001524   
5901139297808997828 1            0   0       1  4.89GB/s  0.010481   
1029742319298993604 1            0   0       1  0.82GB/s  0.001767   
2463791410776694212 1            0   0       1  0.87GB/s  0.001862   
244024996488338884  1            0   0       1  2.07GB/s  0.004440   
...                            ...  ..     ...       ...       ...   
115745911179958724  1            0   0       1  0.11GB/s  0.000231   
268173050679515588  1            0   0       1  0.11GB/s  0.000240   
268173050679450052  1            0   0       1  0.12GB/s  0.000259   
6120539833745067316 2            0   0       1  0.04GB/s  0.000080   
4459700543299833140 2            0   0       1  0.04GB/s  0.000087   

                                              duration           file_id  \
                                     sum           per       sum nunique   
proc_id             io_cat                                                 
9060729382066249156 1       7.637212e+08  1.031003e-01  3.582343       1   
5901139297808997828 1       5.253973e+09  5.182868e-02  1.800849       1   
1029742319298993604 1       8.857923e+08  2.947115e-02  1.024010       1   
2463791410776694212 1       9.331296e+08  2.822791e-02  0.980812       1   
244024996488338884  1       2.225565e+09  2.716527e-02  0.943890       1   
...                                  ...           ...       ...     ...   
115745911179958724  1       1.157430e+08  7.166253e-07  0.000025       1   
268173050679515588  1       1.200833e+08  6.907231e-07  0.000024       1   
268173050679450052  1       1.298198e+08  6.389189e-07  0.000022       1   
6120539833745067316 2       4.023438e+07  3.683857e-07  0.000013       1   
4459700543299833140 2       4.364407e+07  3.396056e-07  0.000012       1   

                           hostname  ... proc_id rank                size  \
                            nunique  ... nunique  max min nunique     fmt   
proc_id             io_cat           ...                                    
9060729382066249156 1             1  ...       1    0   0       1  2.55GB   
5901139297808997828 1             1  ...       1    0   0       1  8.81GB   
1029742319298993604 1             1  ...       1    0   0       1  0.84GB   
2463791410776694212 1             1  ...       1    0   0       1  0.85GB   
244024996488338884  1             1  ...       1    0   0       1  1.96GB   
...                             ...  ...     ...  ...  ..     ...     ...   
115745911179958724  1             1  ...       1   21  21       1  0.00GB   
268173050679515588  1             1  ...       1   21  21       1  0.00GB   
268173050679450052  1             1  ...       1   20  20       1  0.00GB   
6120539833745067316 2             1  ...       1    0   0       1  0.00GB   
4459700543299833140 2             1  ...       1    0   0       1  0.00GB   

                                                        xfer                
                                     per         sum     max    mean   min  
proc_id             io_cat                                                  
9060729382066249156 1       7.431321e-02  2735911684   ~64KB   ~64KB  <4KB  
5901139297808997828 1       2.569977e-01  9461613508  ~256KB  ~256KB  <4KB  
1029742319298993604 1       2.463770e-02   907060444   ~16KB   ~16KB  <4KB  
2463791410776694212 1       2.485947e-02   915225052   ~16KB   ~16KB  <4KB  
244024996488338884  1       5.705919e-02  2100688276   ~64KB   ~64KB  <4KB  
...                                  ...         ...     ...     ...   ...  
115745911179958724  1       7.828129e-08        2882    <4KB    <4KB  <4KB  
268173050679515588  1       7.828129e-08        2882    <4KB    <4KB  <4KB  
268173050679450052  1       7.828129e-08        2882   

time: 63.6 ms (started: 2023-01-04 21:00:27 -08:00)


In [11]:
llc[list(llc.keys())[4]]

acc_pat         app        bw                          \
                  max min nunique       fmt       per           sum   
trange io_cat                                                         
1      1            0   0       4  0.64GB/s  0.003144  6.875395e+08   
       2            0   0       3  0.37GB/s  0.001817  3.974120e+08   
2      1            0   0       4  1.31GB/s  0.006435  1.407302e+09   
3      1            0   0       3  0.89GB/s  0.004378  9.575994e+08   
4      1            0   0       4  1.00GB/s  0.004910  1.073980e+09   
...               ...  ..     ...       ...       ...           ...   
21     2            0   0       1  7.12GB/s  0.034953  7.644525e+09   
23     1            0   0       1  2.91GB/s  0.014302  3.127927e+09   
45     2            0   0       1  7.04GB/s  0.034551  7.556684e+09   
58     2            0   0       1  8.06GB/s  0.039572  8.654787e+09   
25     1            0   0       1  0.25GB/s  0.001242  2.716981e+08   

                   duration            file_id hostname  ... proc_id rank      \
                        per        sum nunique  nunique  ... nunique  max min   
trange io_cat                                            ...                    
1      1       2.871509e-01  10.212173     589       32  ...     202   31   0   
       2       1.200702e-01   4.270150     107       31  ...     160   31   0   
2      1       1.040587e-01   3.700723     247       31  ...      81   25   0   
3      1       6.225113e-02   2.213886     432       32  ...      73    0   0   
4      1       5.614238e-02   1.996636     184       32  ...      64   27   0   
...                     ...        ...     ...      ...  ...     ...  ...  ..   
21     2       3.213944e-05   0.001143       2        2  ...       2    0   0   
23     1       3.143366e-05   0.001118       6        2  ...       2    0   0   
45     2       1.588414e-05   0.000565       1        1  ...       1    0   0   
58     2       1.082562e-05   0.000385       1        1  ...       1    0   0   
25     1       5.961120e-07   0.000021       2        1  ...       1    0   0   

                         size                              xfer                
              nunique     fmt           per         sum     max   mean    min  
trange io_cat                                                                  
1      1           13  6.54GB  1.852604e-01  7021272918  ~256KB  ~16KB   <4KB  
       2           13  1.58GB  4.477658e-02  1697009075  ~256KB  ~64KB   <4KB  
2      1            6  4.85GB  1.374171e-01  5208034870  ~256KB  ~16KB   <4KB  
3      1            1  1.97GB  5.593786e-02  2120015764  ~256KB  ~64KB   <4KB  
4      1            5  2.00GB  5.657982e-02  2144345841  ~256KB  ~16KB   <4KB  
...               ...     ...           ...         ...     ...    ...    ...  
21     2            1  0.01GB  2.305491e-04     8737692   ~16MB  ~16MB  ~16MB  
23     1            1  0.00GB  9.226275e-05     3496710   ~64KB  ~64KB   <4KB  
45     2            1  0.00GB  1.126340e-04     4268771   ~16MB  ~16MB  ~16MB  
58     2            1  0.00GB  8.791923e-05     3332093    ~4MB   ~4MB   ~4MB  
25     1            1  0.00GB  1.519810e-07        5760    <4KB   <4KB   <4KB  

[97 rows x 24 columns]

time: 60.8 ms (started: 2023-01-04 21:00:27 -08:00)


In [12]:
llc[list(llc.keys())[5]]

acc_pat         app        bw            \
                               max min nunique       fmt       per   
file_id             io_cat                                           
7701937538982921416 1            0   0       1  0.71GB/s  0.002825   
7639450094153155784 1            0   0       2  4.89GB/s  0.019438   
7668160541777642696 1            0   0       1  0.72GB/s  0.002851   
7707004088563713224 1            0   0       1  0.82GB/s  0.003277   
7703344913866474696 1            0   0       1  0.87GB/s  0.003452   
...                            ...  ..     ...       ...       ...   
7646768442325907652 2            0   0       1  0.01GB/s  0.000039   
7665908740742232260 2            0   0       1  0.01GB/s  0.000039   
7645924017395775684 2            0   0       1  0.01GB/s  0.000043   
7703344912644749508 2            0   0       1  0.01GB/s  0.000045   
7674352990043551940 2            0   0       1  0.01GB/s  0.000045   

                                              duration           file_id  \
                                     sum           per       sum nunique   
file_id             io_cat                                                 
7701937538982921416 1       7.637212e+08  1.202212e-01  3.582343       1   
7639450094153155784 1       5.253948e+09  6.043569e-02  1.800858       1   
7668160541777642696 1       7.705880e+08  4.832803e-02  1.440075       1   
7707004088563713224 1       8.857923e+08  3.436515e-02  1.024010       1   
7703344913866474696 1       9.331296e+08  3.291545e-02  0.980812       1   
...                                  ...           ...       ...     ...   
7646768442325907652 2       1.050000e+07  2.684750e-07  0.000008       1   
7665908740742232260 2       1.050000e+07  2.684750e-07  0.000008       1   
7645924017395775684 2       1.166667e+07  2.416275e-07  0.000007       1   
7703344912644749508 2       1.217391e+07  2.315597e-07  0.000007       1   
7674352990043551940 2       1.217391e+07  2.315597e-07  0.000007       1   

                           hostname  ... proc_id rank                size  \
                            nunique  ... nunique  max min nunique     fmt   
file_id             io_cat           ...                                    
7701937538982921416 1             1  ...       1    0   0       1  2.55GB   
7639450094153155784 1             2  ...       2    0   0       1  8.81GB   
7668160541777642696 1             2  ...       2    0   0       1  1.03GB   
7707004088563713224 1             1  ...       1    0   0       1  0.84GB   
7703344913866474696 1             1  ...       1    0   0       1  0.85GB   
...                             ...  ...     ...  ...  ..     ...     ...   
7646768442325907652 2             1  ...       1    0   0       1  0.00GB   
7665908740742232260 2             1  ...       1    0   0       1  0.00GB   
7645924017395775684 2             1  ...       1    0   0       1  0.00GB   
7703344912644749508 2             1  ...       1    0   0       1  0.00GB   
7674352990043551940 2             1  ...       1    0   0       1  0.00GB   

                                                        xfer                
                                     per         sum     max    mean   min  
file_id             io_cat                                                  
7701937538982921416 1       8.862726e-02  2735911684   ~64KB   ~64KB  <4KB  
7639450094153155784 1       3.065001e-01  9461616388  ~256KB  ~256KB  <4KB  
7668160541777642696 1       3.594783e-02  1109704616   ~16KB   ~16KB  <4KB  
7707004088563713224 1       2.938336e-02   907060444   ~16KB   ~16KB  <4KB  
7703344913866474696 1       2.964785e-02   915225052   ~16KB   ~16KB  <4KB  
...                                  ...         ...     ...     ...   ...  
7646768442325907652 2       2.721100e-09          84    <4KB    <4KB  <4KB  
7665908740742232260 2       2.721100e-09          84    <4KB    <4KB  <4KB  
7645924017395775684 2       2.721100e-09          84   

time: 67.1 ms (started: 2023-01-04 21:00:27 -08:00)
